## Python Pandas Dataframe

In [ ]:
import streamlit as st
import pandas as pd 
from snowflake.snowpark.functions import col, count, sum, avg, round
import plotly.express as px

from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
session.sql("use database demo").collect()
session.sql("use schema magi_handson").collect()
df = session.table("market_info").to_pandas()
df.head(10)

In [ ]:
df.info()

In [ ]:
st.subheader("시가총액 상위 5개 ETF")

df['MKCAP_TRILLION'] = df['MKCAP']/1e12
top_5 = df.nlargest(5, 'MKCAP')[['BASE_DT', 'ITEM_NM_KOR', 'MKCAP_TRILLION', 'BASE_IDX_NM']]
top_5    

In [ ]:
st.subheader("시가총액 분포")
fig_hist = px.histogram(df, x='MKCAP', nbins=10, 
                       title='시가총액 분포',
                       labels={'MKCAP': '시가총액 (원)', 'count': '빈도'})
st.plotly_chart(fig_hist, use_container_width=True)

In [ ]:
# 매수/매도 금액을 억원 단위로 변환
df['BUY_AMT_BILLION'] = df['SUM_BUY_AMT'] / 1e8
df['SEL_AMT_BILLION'] = df['SUM_SEL_AMT'] / 1e8
df['NET_FLOW'] = df['SUM_BUY_AMT'] - df['SUM_SEL_AMT']
df['NET_FLOW_BILLION'] = df['NET_FLOW'] / 1e8

# 매수금액 상위 ETF
st.subheader("매수금액 상위 5개 ETF")
top_buy = df.nlargest(5, 'SUM_BUY_AMT')[['BASE_DT', 'ITEM_NM_KOR', 'BUY_AMT_BILLION']]
top_buy

In [ ]:
st.header("기준지수별 분석")
    
# 기준지수별 그룹화 분석
index_summary = df.groupby('BASE_IDX_NM').agg({
    'MKCAP': ['count', 'sum', 'mean'],
    'SUM_BUY_AMT': ['sum', 'mean'],
    'SUM_SEL_AMT': ['sum', 'mean']
}).round(2)

index_summary

## Snowpark Pandas Dataframe 

In [ ]:
import modin.pandas as sfpd
import snowflake.snowpark.modin.plugin


In [ ]:
sfpd = sfpd.read_snowflake('snowflake_sample_data.tpch_sf1.nation')
sfpd.describe

In [ ]:
sfpd.loc[sfpd["N_NAME"]=="UNITED STATES","N_NAME"]='USA'
sfpd

In [ ]:
select * from snowflake_sample_data.tpch_sf1.nation;

In [ ]:
session.sql("use database demo").collect()
session.sql("use schema magi_handson").collect()
sfpd_df = session.table("market_info").to_snowpark_pandas()
sfpd_df.head(10)

In [ ]:
sfpd_df.info()

In [ ]:
st.subheader("시가총액 상위 5개 ETF")

sfpd_df['MKCAP_TRILLION'] = sfpd_df['MKCAP']/1e12
top_5 = sfpd_df.nlargest(5, 'MKCAP')[['BASE_DT', 'ITEM_NM_KOR', 'MKCAP_TRILLION', 'BASE_IDX_NM']]
top_5    

In [ ]:
st.subheader("시가총액 분포")
fig_hist2 = px.histogram(sfpd_df, x='MKCAP', nbins=10, 
                       title='시가총액 분포',
                       labels={'MKCAP': '시가총액 (원)', 'count': '빈도'})
st.plotly_chart(fig_hist2, use_container_width=True)

In [ ]:
# 매수/매도 금액을 억원 단위로 변환
sfpd_df['BUY_AMT_BILLION'] = sfpd_df['SUM_BUY_AMT'] / 1e8
sfpd_df['SEL_AMT_BILLION'] = sfpd_df['SUM_SEL_AMT'] / 1e8
sfpd_df['NET_FLOW'] = sfpd_df['SUM_BUY_AMT'] - sfpd_df['SUM_SEL_AMT']
sfpd_df['NET_FLOW_BILLION'] = sfpd_df['NET_FLOW'] / 1e8

# 매수금액 상위 ETF
st.subheader("매수금액 상위 5개 ETF")
top_buy = sfpd_df.nlargest(5, 'SUM_BUY_AMT')[['BASE_DT', 'ITEM_NM_KOR', 'BUY_AMT_BILLION']]
top_buy

In [ ]:
st.header("기준지수별 분석")
    
# 기준지수별 그룹화 분석
index_summary = sfpd_df.groupby('BASE_IDX_NM').agg({
    'MKCAP': ['count', 'sum', 'mean'],
    'SUM_BUY_AMT': ['sum', 'mean'],
    'SUM_SEL_AMT': ['sum', 'mean']
}).round(2)

index_summary

## Snowpark Dataframe 

In [ ]:
session.sql("use database demo").collect()
session.sql("use schema magi_handson").collect()
snowpark_df = session.table("market_info")
snowpark_df.limit(10)

In [ ]:
snowpark_df.describe

In [ ]:
st.subheader("시가총액 상위 5개 ETF")
top_5_snowpark = (snowpark_df
        .with_column("MKCAP_TRILLION", col("MKCAP") / 1e12)
        .order_by(col("MKCAP").desc())
        .limit(5)
        .select("BASE_DT", "ITEM_NM_KOR", "MKCAP_TRILLION", "BASE_IDX_NM")
)
top_5_snowpark

In [ ]:
st.subheader("시가총액 분포")
fig_hist3 = px.histogram(snowpark_df, x='MKCAP', nbins=10, 
                       title='시가총액 분포',
                       labels={'MKCAP': '시가총액 (조원)', 'count': '빈도'})
st.plotly_chart(fig_hist3, use_container_width=True)

In [ ]:
st.subheader("매수금액 상위 5개 ETF")
df_with_calculations = (snowpark_df
        .with_column("BUY_AMT_BILLION", col("SUM_BUY_AMT") / 1e8)
        .with_column("SEL_AMT_BILLION", col("SUM_SEL_AMT") / 1e8)
        .with_column("NET_FLOW", col("SUM_BUY_AMT") - col("SUM_SEL_AMT"))
        .with_column("NET_FLOW_BILLION", col("NET_FLOW") / 1e8)
    )
    
# 매수금액 상위 5개 ETF
top_buy_snowpark = (df_with_calculations
    .order_by(col("SUM_BUY_AMT").desc())
    .limit(5)
    .select("BASE_DT", "ITEM_NM_KOR", "BUY_AMT_BILLION")
)

top_buy_snowpark

In [ ]:
st.header("기준지수별 분석")

index_summary = (snowpark_df
    .group_by("BASE_IDX_NM")
    .agg(
        count(col("MKCAP")).alias("MKCAP_count"),
        sum(col("MKCAP")).alias("MKCAP_sum"),
        avg(col("MKCAP")).alias("MKCAP_mean"),
        sum(col("SUM_BUY_AMT")).alias("BUY_sum"),
        avg(col("SUM_BUY_AMT")).alias("BUY_mean"),
        sum(col("SUM_SEL_AMT")).alias("SEL_sum"),
        avg(col("SUM_SEL_AMT")).alias("SEL_mean")
    )
    .select(
        col("BASE_IDX_NM"),
        round(col("MKCAP_count"), 2).alias("MKCAP_count"),
        round(col("MKCAP_sum"), 2).alias("MKCAP_sum"),
        round(col("MKCAP_mean"), 2).alias("MKCAP_mean"),
        round(col("BUY_sum"), 2).alias("BUY_sum"),
        round(col("BUY_mean"), 2).alias("BUY_mean"),
        round(col("SEL_sum"), 2).alias("SEL_sum"),
        round(col("SEL_mean"), 2).alias("SEL_mean")
    )
)

index_summary

## 성능 비교  - Native Pandas

In [ ]:
# pd_df = session.table("snowflake_sample_data.tpch_sf100.orders").to_pandas()
pd_df = session.table("snowflake_sample_data.tpch_sf10.orders").to_pandas()

pd_df_sales = pd_df.groupby('O_CUSTKEY').agg({
    'O_TOTALPRICE': ['count', 'sum', 'mean'],
}).round(2)

pd_df_sales.head(10)

## 성능 비교  - Snowpark Pandas

In [ ]:
# sfpd_df = session.table("snowflake_sample_data.tpch_sf100.orders").to_snowpark_pandas()
sfpd_df = session.table("snowflake_sample_data.tpch_sf10.orders").to_snowpark_pandas()

sfpd_df_sales = sfpd_df.groupby('O_CUSTKEY').agg({
    'O_TOTALPRICE': ['count', 'sum', 'mean'],
}).round(2)

sfpd_df_sales.head(10)

## 성능 비교  - Snowpark Dataframe

In [ ]:
# sf_df = session.table("snowflake_sample_data.tpch_sf100.orders")
sf_df = session.table("snowflake_sample_data.tpch_sf10.orders")

sf_df_sales = sf_df.group_by('O_CUSTKEY').agg(
    count(col("O_TOTALPRICE")).alias("O_TOTALPRICE_count"),
    sum(col("O_TOTALPRICE")).alias("O_TOTALPRICE_sum"),
    avg(col("O_TOTALPRICE")).alias("O_TOTALPRICE_mean")
)

sf_df_sales.limit(10)

### Using pandas on Snowflake with Snowpark DataFrames
https://docs.snowflake.com/en/developer-guide/snowpark/python/pandas-on-snowflake#using-pandas-on-snowflake-with-snowpark-dataframes

### Snowpark DataFrames vs Snowpark pandas DataFrame: Which should I choose?
https://docs.snowflake.com/en/developer-guide/snowpark/python/working-with-dataframes#snowpark-dataframes-vs-snowpark-pandas-dataframe-which-should-i-choose